In [11]:
import os
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/workspace')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
from datetime import datetime, timedelta
def get_avg_time(time_1, time_2):
    # 将时间字符串转换为时间间隔
    format_string = '%H:%M:%S.%f'
    time1_obj = datetime.strptime(time_1, format_string)
    time2_obj = datetime.strptime(time_2, format_string)

    # 计算时间间隔的总和
    total_time = time1_obj - datetime.min + (time2_obj - datetime.min)

    # 计算平均时间间隔
    average_time = total_time / 2

    # 将平均时间间隔格式化为相同的时间字符串格式
    average_time_str = (datetime.min + average_time).time().strftime(format_string)
    return average_time_str

In [18]:
def getTimeAndText(ass_path):
    with open(ass_path, 'r', encoding='utf-16') as file:
        lines = file.readlines()
    keyword = "zhengwen"
    avg_time_list = []
    text_list = []
    for line in lines:
        if line.startswith('Dialogue') and keyword in line:
            content = line.split(":", 1)[1].strip()
            line_list = content.split(",")
            avg_time = get_avg_time(line_list[1], line_list[2])
            avg_time_list.append(avg_time)
            text_list.append(line_list[-1])
    return avg_time_list, text_list

In [23]:
import json
import subprocess

def extract_frame_at_time(video_path, output_path, time_stamp):
    ffmpeg_cmd = [
        'ffmpeg',
        '-ss', time_stamp,
        '-i', video_path,
        '-vframes', '1',
        output_path
    ]

    subprocess.run(ffmpeg_cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

# for id in [str(x).zfill(2) for x in range(15, 29)]:
data = []
ass_path = f"Haruhi_all/[CASO][Suzumiya_Haruhi_no_Yuuutsu][27][BDRIP][1920x1080][x264_FLAC_2][B3301EC5].SC.ass"
video_path = f"Haruhi_all/[CASO][Suzumiya_Haruhi_no_Yuuutsu][27][BDRIP][1920x1080][x264_FLAC_2][B3301EC5].mkv"

avg_time_list, text_list = getTimeAndText(ass_path)

for i in range(len(avg_time_list)):
    time_stamp = avg_time_list[i]
    output_path = f'haruhi_data/images_28/{text_list[i]}{avg_time_list[i]}.jpg'
    extract_frame_at_time(video_path, output_path, time_stamp)
    data.append({"text":str(text_list[i]), "timestamp":str(avg_time_list[i]), "path":f"{text_list[i]}{avg_time_list[i]}.jpg"})

with open(f'haruhi_data/info_28.json', 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)